# EDA

## ane.energy data files

In [2]:
import pandas as pd
import vaex

In [ ]:
info = pd.DataFrame(data=[['crossborder_flows', 
                           'dt_start_utc, power_mw, from_country, to_country', 
                           '2016-12-31 23:00', '2019-09-20 02:00', '1h', 
                           'ac', (142816, 4), '6.6', 
                           '4h behind live data'],
                          ['epex_da_de', 
                           'dt_start_utc, sechs_h_regelung, epex_da_de_eur_mwh', 
                           '2004-12-31 23:00', '2021-07-13 21:00', '1h', 
                           '?', (144911, 3), '4.7', 
                           ''],
                          ['epex_da_prognosis', 
                           'predicted_at, predicted_for, prediction',
                           '2018-12-26 23:00','2021-07-01 22:00','1h',
                           'fc',(483097, 3),'24.4',''],
                          ['es_forecast', 
                           'dt_start_utc, power_mw, carrier, type, area, version_utc',
                           '2016-12-31 23:00','2020-02-13 23:45','15min',
                           'fc',(36876431, 6),'2700',''],
                          ['imbalance_de', 
                           'time, val1, val2',
                           '2013-12-31 23:00','2021-04-29 23:45','15min',
                           'ac',(257009, 3),'10.1',
                           'end = last day fully covered; published for past month'],
                          ['power_ac', 
                           'dt_start_utc, power_act_21, 24, 47, 84, 176, 179, 183, 196, 202, 206, 222, 233, 291, 292, 293, 308, 336',
                           '2019-06-30 22:00','2021-04-30 23:45','15min',
                           'ac',(64328,18),'5.1',
                           'live data'],
                          ['power_fc', 
                           'dt_start_utc, power_fc_21, 24, 47, 48, 51, 176, 179, 183, 196, 202, 206, 222, 233, 291, 292, 293, 308, 322, 325, 327, 336, 339',
                           '2019-06-13 07:00','2021-04-30 23:45','15min',
                           'fc',(66020, 23),'5.4',
                           'meaning of timestamp unclear'],
                          ['regelleistung_aggr', 
                           'date_start, date_end, product, reserve_type, 6 total capacity values, 7 German capacity values',
                           '2019-01-01','2021-03-19','1d',
                           'ac',(16068, 17),'1.5',
                           'available for past month; date_start == date_end'],
                          ['regelleistung_demand', 
                           'date_start, date_end, product, total_demand, germany_block_demand, reserve_type',
                           '2019-01-01','2021-03-18','1d',
                           'ac',(16188, 6),'0.7',
                           'available for past month; date_start == date_end'],
                          ['solar_mw', 
                           'dt_start_utc, fiftyhertz, tennet, amprion, transnetbw, nrv',
                           '2011-12-31 23:00','2021-07-11 21:00','1h',
                           'ac',(83519, 6),'3.6',
                           'available for past month'],
                          ['wind_offshore_mw', 
                           'dt_start_utc, fiftyhertz, tennet, sum',
                           '2012-12-31 23:00','2021-07-11 21:00','1h',
                           'ac',(74735, ),'4.4',
                           'available for past month'],
                          ['wind_onshore_mw', 
                           'dt_start_utc, fiftyhertz, tennet, amprion, transnetbw, nrv',
                           '2011-12-31 23:00','2021-07-01 21:00','1h',
                           'ac',(83278, 6),'5.5',
                           'available for past month; meaning of values: 4 control zones + sum?'],
                          ['wind_speed', 
                           'dt_start_utc, voronoi_area_id, windspeed',
                           '2018-12-31 23:00','2020-09-30 23:45','15min','2h delay',(6548085, 3),'187.6','']
                         ],
                   columns=['filename', 'data', 'start', 'end', 'timestep', 'ac/fc', 'shape', 'filesize_MB', 'notes'])

In [4]:
info.to_csv('info.csv')

### `imbalance_de`

15min data only until 2021-04-30 21:45, after that only occasional data until 2021-05-22 22:00 (skipped latter)

Actual imbalance power can be taken from 
- SMARD > Marktdaten > Oberkategorei: Systemstabilität > Datenkategorie Ausgleichsenergie > Land: Deutschland or 
- ENTSOE > Balancing > Imbalance > Germany

(see below) with a delay of 1h (4 time steps)

May alsom be updated from that source

### `es_forecast`

turn into hdf5 format

In [5]:
esf = vaex.from_csv('data/es_forecast.csv', nrows = 30)
for i, df in enumerate(vaex.from_csv('data/es_forecast.csv', chunk_size=100_000)):
    if i%50 == 0: print(i)
    df.export_hdf5(f'data/es_forecast/es_forecast_{i:02}.hdf5')
df = vaex.open('data/es_forecast/es_forecast*')
df.export_hdf5('data/es_forecast.hdf5')

FileNotFoundError: [Errno 2] Failed to open local file 'data/es_forecast.csv'. Detail: [errno 2] No such file or directory

columns: `dt_start_utc`, `power_mw`, `carrier`, `type`, `area`, `version_utc`

carrier.value_counts(): Gesamt 15896126, Wind Offshore 7010591, Solar 6993803, Wind Onshore 6975911, dtype int64
 
type.value_counts(): Renewables Forecast 28033409, Load Forecast 8843022, dtype int64

area.value_counts(): 50Hertz 10652224, TTG (=Tennet) 10608236, DE 10435444, DK1 2594607, DK 2585920, dtype int64

split by carrier and type into ordinary pandas data frames, removed duplicates and inconsistent entries (different `power_mw` values for the same `dt_start_utc`, `carrier`, `type` and `area`), sorted the time stamps, and saved as csv files:

In [ ]:
esf = vaex.open('data_raw/es_forecast.hdf5')
esf['dt_start_utc'] = esf['dt_start_utc'].astype('datetime64')

carriers = esf.carrier.unique()
types = esf['type'].unique()

for c in carriers:
    for t in types:
        print(c, t)
        df = esf[esf.carrier == c][esf.type == t].to_pandas_df(['dt_start_utc', 'power_mw', 
                                                                'area', 'version_utc'])
        df.drop_duplicates(subset=['dt_start_utc', 'area'], inplace=True)
        if len(df) > 0:
            df.sort_values('dt_start_utc', inplace=True)
            df.set_index('dt_start_utc', inplace=True)
            filename = c.replace(' ','_')
            if c == 'Gesamt':
                filename = 'total' + '_' + t[:t.find(' ')]
            filename = 'es_fc_' + filename.lower()
            print(filename, len(df))
            df.to_csv('data_raw/'+filename+'.csv')

After 2019-09-23, the `version_utc` time stamp is older than `dt_start_utc`. Therefore, we may assume that the values are always known at the time of prediction. So, we dropped `version_utc`, remove occasional further duplicates due to different forecasts (later on one may collect these forecasts into a dict or take the last one or ...) and casted each of these ordinary data frames into time series by means of cast function with method `column`, filled nans by method `ffill`

written to files `es_fc_solar_ts`, `es_fc_wind_offshore_ts`, `es_fc_wind_onshore_ts`, `es_fc_total_renewables_ts`, `es_fc_total_load_ts`

entries of `es_fc_total_load_ts` coincide with entries from solar, wind_offshore or wind_onshore at many places (146 000 from 158 000) but randomly chosen --> ignore

### `regelleistung_demand` and `regelleistung_aggr`

Columns `date_start` and `date_end` have the same entries --> drop `date_end`

Column `reserve_type` has values MRL (Minutenreserveleistung, Tertiärregelung) and SRL (Sekundärregelleistung). For MRL, `total_demand_mw` and `germany_block_demand_mw` have the same entries. For SRL, this holds until 2020-01-31, after that the difference first - second is 200, independent of `product`. --> turn into columns MRL, SRL_total, SRL_germany

Column `product` contains `NEG_00_04` valid from 00:00 to 04:00, `NEG_04_08` valid from 04:00 to 08:00 etc and the same with `NEG` replaced by `POS`. Split accordingly so that `NEG` and `POS` combine with the remaining columns to 52 new columns and the timestamp runs in 4h steps: 

In [ ]:
filename = 'regelleistung_aggr'# and 'regelleistung_demand'
df = pd.read_csv('data/'+filename+'_ts.csv')#, parse_dates=['dt_start_utc'])#.iloc[:1000]

hours = ['00_04', '04_08', '08_12', '12_16', '16_20', '20_24']
start = df.date_start.iloc[0]
end = df.date_start.iloc[-1]
dfl = []
for hour in hours:
    cols = [x for x in df.columns if hour in x]
    dfi = df[['date_start'] + cols]
    dfi['date_start'] = dfi.date_start + ' ' + hour[:2] + ':00:00'
    dfi.rename(columns = {'date_start':'dt_start_utc'}, inplace=True)
    dfi['dt_start_utc'] = dfi.dt_start_utc.astype('datetime64')
    dfi.set_index('dt_start_utc', drop=True, inplace=True)
    if hour != '00_04':
        firstrow = pd.DataFrame(data=[[start+' 00:00:00'] + [0 for col in cols]], columns=['dt_start_utc']+cols)
        firstrow['dt_start_utc'] = firstrow.dt_start_utc.astype('datetime64')
        firstrow.set_index('dt_start_utc', drop=True, inplace=True)
        dfi = pd.concat([firstrow, dfi], axis=0)
    if hour != '20_24':
        lastrow = pd.DataFrame(data=[[end+' 20:00:00'] + [0 for col in cols]], columns=['dt_start_utc']+cols)
        lastrow['dt_start_utc'] = lastrow.dt_start_utc.astype('datetime64')
        lastrow.set_index('dt_start_utc', drop=True, inplace=True)
        dfi = pd.concat([dfi, lastrow], axis=0)
    dfi.columns = [x.replace('NEG_'+hour, 'neg') for x in dfi.columns]
    dfi.columns = [x.replace('POS_'+hour, 'pos') for x in dfi.columns]
    dfi.columns = [x.lower() for x in dfi.columns]
    dfl.append(dfi)

dft = align(*dfl, value=0)

cols = [x[2:] for x in dft.columns if '1_' in x]

for col in cols:
    nrcols = [str(i)+'_'+col for i in range(1,7)]
    dft[col] = dft[nrcols].sum(axis=1)

dft[cols].to_csv('data/'+filename+'_4h.csv')

### `wind_speed`

Does not contain any data for `voronoi_area_id = 3`.

Casted into time series and filled missing values first forward then backward (the latter because there were areas without data at starting time).

Note: the output of the cell the function `cast` was run in was partly missing. Yet the function seems to have processed all data properly. At least no data point was missing. Maybe run again in chunks of 1 000 000 and concatenate afterwards.

### `epex_da_prognosis`

first changed dt_fc_utc into number of days in advance the prognosis was made

```df['dt_fc_utc'] = (df.dt_start_utc.dt.date - df.dt_fc_utc.dt.date).astype('int')//86400000000000```

then casted into a dictionary aggregated time series. 

Read off last and second last prognosis:

In [ ]:
df2 = pd.read_csv('data/epex_da_prognosis_ts.csv', parse_dates=True, index_col='dt_start_utc')
df2['forecasts'] = [ast.literal_eval(x) for x in df2.da_price_eur_mwh]
df2['newest_fc'] = [x[str(min(int(y) for y in x.keys()))] 
                    #if '1' in x.keys() 
                    #else x['2'] if '2' in x.keys() 
                    #else x['3'] if '3' in x.keys()                     
                    #else None 
                    for x in df2.forecasts]
df2['second_newest_fc'] = [x[sorted(x.keys())[1]] 
                    if len(x.keys()) > 1 else None
                    #else x['2'] if '2' in x.keys() 
                    #else x['3'] if '3' in x.keys()                     
                    #else None 
                    for x in df2.forecasts]

df2.drop(['forecasts'], axis=1, inplace=True)

### `crossborder_flows`

available only after 4h and poorly correlated --> omit

## SMARD data

Source: https://www.smard.de/home/downloadcenter/download-marktdaten 

The time stamps are Central European Time (UTC+1h during winter time and UTC+2h during summer time). They have been transformed to UTC.

The delays given here are estimates from occasional observation of the website. They may depend on daytime, and they may be longer in the night. (!) Have to be checked carefully; deadlines may be taken from https://www.smard.de/home/datennutzung; generally a workaround for the situation where life values are accidentally missing is needed.

Reshaped via following routine (for the example of consumption_ac).

In [ ]:
def time2idx(df):
    df['Datum'] = [x[6:]+'-'+x[3:5]+'-'+x[:2] for x in df.Datum]
    df['dt_start_utc'] = df.Datum + ' ' + df.Uhrzeit
    df['dt_start_utc'] = (df.Datum + ' ' + df.Uhrzeit).astype('datetime64')
    df.set_index('dt_start_utc', inplace=True)
    return df.drop(['Datum', 'Uhrzeit'], axis=1)

In [ ]:
dfl = [time2idx(pd.read_csv('smard/consumption/consumption_ac/consumption_ac_2015.csv', sep=';')), 
       time2idx(pd.read_csv('smard/consumption/consumption_ac/consumption_ac_2016.csv', sep=';')), 
       time2idx(pd.read_csv('smard/consumption/consumption_ac/consumption_ac_2017.csv', sep=';')), 
       time2idx(pd.read_csv('smard/consumption/consumption_ac/consumption_ac_2018.csv', sep=';')), 
       time2idx(pd.read_csv('smard/consumption/consumption_ac/consumption_ac_2019.csv', sep=';')), 
       time2idx(pd.read_csv('smard/consumption/consumption_ac/consumption_ac_2020.csv', sep=';')), 
       time2idx(pd.read_csv('smard/consumption/consumption_ac/consumption_ac_2021.csv', sep=';'))
      ]

In [ ]:
for i in range(len(dfl)):
    print(dfl[i].columns)

In [ ]:
for i in range(len(dfl)):
    dfl[i].rename(columns={'Gesamt_(Netzlast)[MWh]_real_verb':'consumption_ac_mwh',
                      'Residuallast[MWh]_real_verb':'residual_ac_mwh',
                      'Pumpspeicher[MWh]_real_verb':'pump_storage_ac_mwh'
                     }, inplace=True)

In [ ]:
df = pd.concat(dfl, axis=0)

In [ ]:
df.to_csv('data/smard_consumption_ac.csv')

### `smard_generation_ac`

Actual generation in Germany. 

Source: Oberkategorie: Stromerzeugung > Datenkategorie: Realisierte Erzeugung > Land: Deutschland

Delay: 2h (8 time steps)

Columns renamer:

In [ ]:
for i in range(len(dfl)):
    dfl[i].rename(columns={'Biomasse[MWh]_real_erzeugt':'gen_biomass_ac_mwh',
                    'Wasserkraft[MWh]_real_erzeugt':'gen_waterpower_ac_mwh',
                    'Wind Offshore[MWh]_real_erzeugt':'gen_wind_offshore_ac_mwh', 
                    'Wind Onshore[MWh]_real_erzeugt':'gen_wind_onshore_ac_mwh',
                    'Photovoltaik[MWh]_real_erzeugt_real_erzeugt':'gen_solar_ac_mwh',
                    'Sonstige Erneuerbare[MWh]_real_erzeugt':'gen_other_renewables_ac_mwh',
                    'Kernenergie[MWh]_real_erzeugt':'gen_nuclear_ac_mwh', 
                    'Braunkohle[MWh]_real_erzeugt':'gen_lignite_ac_mwh',
                    'Steinkohle[MWh]_real_erzeugt':'gen_anthracite_ac_mwh',
                    'Erdgas[MWh]_real_erzeugt':'gen_gas_ac_mwh',
                    'Pumpspeicher[MWh]_real_erzeugt':'gen_pumped_storage_ac_mwh',
                    'Sonstige Konventionelle[MWh]_real_erzeugt':'gen_other_conventional_ac_mwh'
                   }, inplace=True)

### `smard_generation_fc`

Prognosticated generation in Germany

Source: Oberkategorie: Stromerzeugung > Datenkategorie: Prognostizierte Erzeugung > Land: Deutschland

Delay: 0h

Column renamer:

### `smard_consumption_ac`

Actual consumption in Germany

Source: Oberkategorie: Stromverbrauch > Datenkategorie: Realisierter Stromverbrauch > Land: Deutschland

Delay: 2h (8 time steps)

Column renamer:

In [ ]:
for i in range(len(dfl)):
    dfl[i].rename(columns={'Gesamt_(Netzlast)[MWh]_real_verb':'consumption_ac_mwh',
                      'Residuallast[MWh]_real_verb':'residual_ac_mwh',
                      'Pumpspeicher[MWh]_real_verb':'pump_storage_ac_mwh'
                     }, inplace=True)

### `smard_consumption_fc`

Prognosticated consumption in Germany

Source: Oberkategorie: Stromverbrauch > Datenkategorie: Prognostizierter Stromverbrauch > Land: Deutschland

Delay: 0h

Column renamer:

In [ ]:
for i in range(len(dfl)):
    dfl[i].rename(columns={'Gesamt[MWh]_prog_verb':'consumption_fc_mwh',
                           'Prognostizierte Residuallast[MWh]_prog_verb':'residual_fc_mwh'
                          }, inplace=True)

### `smard_mrl`

Minutenreserve

Source: Oberkategorie: Systemstabilität > Datenkategorie: Minutenreserve > Land: Deutschland

Delay: 
- 'mrl_energy_pos_mwh', 'mrl_energy_neg_mwh', mrl_energy_price_pos_eur_mwh', 'mrl_energy_price_neg_eur_mwh': unclear; assume 1h (4 time steps) like for `smard_srl`
- 'mrl_capacity_pos_mwh', 'mrl_capacity_neg_mwh', 'mrl_capacity_price_pos_eur_mwh', 'mrl_capacity_price_neg_eur_mwh': 0h
                           
Column renamer:

In [ ]:
for i in range(len(dfl)):
    dfl[i].rename(columns={'Abgerufene Menge (+)[MWh]_min_res':'mrl_energy_pos_mwh',
                           'Abgerufene Menge (-)[MWh]_min_res':'mrl_energy_neg_mwh',
                           'Arbeitspreis (+)[€/MWh]_min_res':'mrl_energy_price_pos_eur_mwh',
                           'Arbeitspreis (-)[€/MWh]_min_res':'mrl_energy_price_neg_eur_mwh',
                           'Vorgehaltene Menge (+)[MW]_min_res':'mrl_capacity_pos_mwh',
                           'Vorgehaltene Menge (-)[MW]_min_res':'mrl_capacity_neg_mwh',
                           'Leistungspreis (+)[€/MW]_min_res':'mrl_capacity_price_pos_eur_mwh',
                           'Leistungspreis (-)[€/MW]_min_res':'mrl_capacity_price_neg_eur_mwh'
                          }, inplace=True)

### `smard_prl`

Primärregelreserve

Source: Oberkategorie: Systemstabilität > Datenkategorie: Primärregelreserve > Land: Deutschland

Delay: 0h

Column renamer:

### `smard_srl`

Sekundärregelreserve

Source: Oberkategorie: Systemstabilität > Datenkategorie: Sekundärregelreserve > Land: Deutschland

Delay: 
- 'srl_energy_pos_mwh', 'srl_energy_neg_mwh', 'srl_energy_price_pos_eur_mwh', 'srl_energy_price_neg_eur_mwh': 1h (4 time steps)
- 'srl_capacity_pos_mwh', 'srl_capacity_neg_mwh', 'srl_capacity_price_pos_eur_mwh', 'srl_capacity_price_neg_eur_mwh': 0h

In [ ]:
for i in range(len(dfl)):
    dfl[i].rename(columns={'Abgerufene Menge (+)[MWh]_sek_rresv':'srl_energy_pos_mwh',
                           'Abgerufene Menge (-)[MWh]_sek_rresv':'srl_energy_neg_mwh',
                           'Arbeitspreis (+)[€/MWh]_sek_rresv':'srl_energy_price_pos_eur_mwh',
                           'Arbeitspreis (-)[€/MWh]_sek_rresv':'srl_energy_price_neg_eur_mwh',
                           'Vorgehaltene Menge (+)[MW]_sek_rresv':'srl_capacity_pos_mwh',
                           'Vorgehaltene Menge (-)[MW]_sek_rresv':'srl_capacity_neg_mwh',
                           'Leistungspreis (+)[€/MW]_sek_rresv':'srl_capacity_price_pos_eur_mwh',
                           'Leistungspreis (-)[€/MW]_sek_rresv':'srl_capacity_price_neg_eur_mwh'
                          }, inplace=True)

### `imbalance_at`

same as `entsoe_imbalance_at`

Source: Oberkategorie: Systemstabilität > Datenkategorie: Ausgleichsenergie > Land: Österreich

Delay: 2.5h (10 time steps)

Combined from `imbalance_export_at` and `imbalance_import_at` via

In [ ]:
for i in range(len(dfl)):
    dfl[i].rename(columns={'Österreich[MWh]_exp_regel':'imbalance_export_at_mwh'
                     }, inplace=True)

In [274]:
dfl = [time2idx(pd.read_csv('data_raw/smard/stability/imbalance_export_at/imbalance_export_at_2016.csv', sep=';')), 
       time2idx(pd.read_csv('data_raw/smard/stability/imbalance_export_at/imbalance_export_at_2017_2018.csv', sep=';')), 
       time2idx(pd.read_csv('data_raw/smard/stability/imbalance_export_at/imbalance_export_at_2019_2020.csv', sep=';')), 
       time2idx(pd.read_csv('data_raw/smard/stability/imbalance_export_at/imbalance_export_at_2021.csv', sep=';')), 
      ]

In [275]:
for i in range(len(dfl)):
    print(dfl[i].columns)

Index(['Österreich[MWh]_exp_regel'], dtype='object')
Index(['Österreich[MWh]_exp_regel'], dtype='object')
Index(['Österreich[MWh]_exp_regel'], dtype='object')
Index(['Österreich[MWh]_exp_regel'], dtype='object')


In [276]:
for i in range(len(dfl)):
    dfl[i].rename(columns={'Österreich[MWh]_exp_regel':'imbalance_export_at_mwh'
                     }, inplace=True)

In [277]:
df1 = pd.concat(dfl, axis=0)

In [279]:
for i in range(len(dfl)):
    print(dfl[i].columns)

Index(['Österreich[MWh]_imp_regel'], dtype='object')
Index(['Österreich[MWh]_imp_regel'], dtype='object')
Index(['Österreich[MWh]_imp_regel'], dtype='object')
Index(['Österreich[MWh]_imp_regel'], dtype='object')


In [280]:
for i in range(len(dfl)):
    dfl[i].rename(columns={'Österreich[MWh]_imp_regel':'imbalance_import_at_mwh'
                     }, inplace=True)

In [281]:
df2 = pd.concat(dfl, axis=0)

In [282]:
df = align(df1, df2, prefixes=('',''))

In [283]:
df.to_csv('data/smard_imbalance_at.csv')

## ENTSOE data

From https://transparency.entsoe.eu > Balancing > Imbalance

In [278]:
dfl = [time2idx(pd.read_csv('data_raw/smard/stability/imbalance_import_at/imbalance_import_at_2016.csv', sep=';')), 
       time2idx(pd.read_csv('data_raw/smard/stability/imbalance_import_at/imbalance_import_at_2017_2018.csv', sep=';')), 
       time2idx(pd.read_csv('data_raw/smard/stability/imbalance_import_at/imbalance_import_at_2019_2020.csv', sep=';')), 
       time2idx(pd.read_csv('data_raw/smard/stability/imbalance_import_at/imbalance_import_at_2021.csv', sep=';')), 
      ]

Timestamp correction due to forgotten option dayfirst=True (example pl):

In [6]:
country = 'pl'
df = pd.read_csv('data/entsoe_imbalance_'+country+'.csv')
df.rename(columns = {'dt_start_utc':'old'}, inplace=True)
df['new'] = [x[:4]+'-'+x[8:10]+'-'+x[5:7]+x[10:] for x in df.old]
df['dt_start_utc'] = [0 for x in df.index]
for idx in df.index:
    try: df['dt_start_utc'][idx] = pd.to_datetime(df.new[idx])
    except: df['dt_start_utc'][idx] = pd.to_datetime(df.old[idx])
df.drop(['old', 'new'], axis=1, inplace=True)
df.set_index('dt_start_utc', inplace=True)
df.to_csv('data/entsoe_imbalance_'+country+'.csv')

### `imbalance_at`

Same as `smard_imbalance_at`


Delay: 2.5h (10 time steps)

Preparation:

In [132]:
df = pd.read_csv('entsoe/imbalance_at.csv', parse_dates=['dt_start_utc'], index_col='dt_start_utc')

In [119]:
print(df.columns)

Index(['+ Imbalance Price [EUR/MWh] - SCA|BE',
       '- Imbalance Price [EUR/MWh] - SCA|BE', 'Status',
       'Total Imbalance [MWh] - SCA|BE', 'Difference', 'Situation',
       'Status.1'],
      dtype='object')


In [133]:
(df['+ Imbalance Price [EUR/MWh] - SCA|AT']==df['- Imbalance Price [EUR/MWh] - SCA|AT']).value_counts()

True    20089
dtype: int64

In [134]:
df.drop(['Status.1', '- Imbalance Price [EUR/MWh] - SCA|AT', 'Difference'], axis=1, inplace=True)

In [135]:
df.rename(columns={'+ Imbalance Price [EUR/MWh] - SCA|AT':'rebap_at_eur_mwh', 
                   'Total Imbalance [MWh] - SCA|AT':'imbalance_power_at_mwh', 
                   'Status':'status',
                   'Situation':'sign',
                     }, inplace=True)

In [136]:
df['sign'] = [-1 if x=='Deficit' else 1 for x in df.sign]
df['imbalance_power_at_mwh'] = df.imbalance_power_at_mwh * df.sign
df.drop('sign', axis=1, inplace=True)

In [138]:
df.to_csv('data/entsoe_imbalance_at.csv')

## `imbalance_be`

Delay: 2h (8 time steps)

Preparation:

In [125]:
df = pd.read_csv('entsoe/imbalance_be.csv', parse_dates=['dt_start_utc'], index_col='dt_start_utc')

In [119]:
print(df.columns)

Index(['+ Imbalance Price [EUR/MWh] - SCA|BE',
       '- Imbalance Price [EUR/MWh] - SCA|BE', 'Status',
       'Total Imbalance [MWh] - SCA|BE', 'Difference', 'Situation',
       'Status.1'],
      dtype='object')


In [120]:
(df['+ Imbalance Price [EUR/MWh] - SCA|BE']==df['- Imbalance Price [EUR/MWh] - SCA|BE']).value_counts()

True    20114
dtype: int64

In [126]:
df.drop(['Status.1', '- Imbalance Price [EUR/MWh] - SCA|BE', 'Difference'], axis=1, inplace=True)

In [127]:
df.rename(columns={'+ Imbalance Price [EUR/MWh] - SCA|BE':'rebap_be_eur_mwh', 
                   'Total Imbalance [MWh] - SCA|BE':'imbalance_power_be_mwh', 
                   'Status':'status',
                   'Situation':'sign',
                     }, inplace=True)

In [128]:
df['sign'] = [-1 if x=='Deficit' else 1 for x in df.sign]
df['imbalance_power_be_mwh'] = df.imbalance_power_be_mwh * df.sign
df.drop('sign', axis=1, inplace=True)

In [130]:
df.to_csv('data/entsoe_imbalance_be.csv')

## `imbalance_dk`

Delay: 4h (16 time steps)

Preparation:

In [104]:
df = pd.read_csv('entsoe/imbalance_dk.csv', parse_dates=['dt_start_utc'], index_col='dt_start_utc')

In [105]:
print(df.columns)

Index(['Generation / + Imbalance price [EUR/MWh] - MBA|DK2',
       'Generation / - Imbalance price [EUR/MWh] - MBA|DK2',
       'Price for consumption [EUR/MWh] - MBA|DK2', 'Status',
       'Total Imbalance [MWh] - MBA|DK2', 'Difference', 'Situation',
       'Status.1'],
      dtype='object')


In [109]:
(df['Generation / + Imbalance price [EUR/MWh] - MBA|DK2']==df['Generation / - Imbalance price [EUR/MWh] - MBA|DK2']).value_counts()

False    3078
True     1950
dtype: int64

In [114]:
df.drop(['Status.1', 'Difference'], axis=1, inplace=True)

In [110]:
df.rename(columns={'Generation / + Imbalance price [EUR/MWh] - MBA|DK2':'imbalance_price_dk_pos_eur_mwh', 
                   'Generation / - Imbalance price [EUR/MWh] - MBA|DK2':'imbalance_price_dk_neg_eur_mwh', 
                   'Price for consumption [EUR/MWh] - MBA|DK2':'rebap_dk_eur_mwh',
                   'Total Imbalance [MWh] - MBA|DK2':'imbalance_power_dk_mwh', 
                   'Status':'status',
                   'Situation':'sign',
                     }, inplace=True)

In [112]:
df['sign'] = [-1 if x=='Deficit' else 1 for x in df.sign]
df['imbalance_power_dk_mwh'] = df.imbalance_power_dk_mwh * df.sign
df.drop('sign', axis=1, inplace=True)

In [116]:
df.to_csv('data/entsoe_imbalance_dk.csv')

## `imbalance_fr`

Not used, because past values are not available before July 25.

Preparation

In [139]:
df = pd.read_csv('entsoe/imbalance_fr.csv', parse_dates=['dt_start_utc'], index_col='dt_start_utc')

In [140]:
print(df.columns)

Index(['+ Imbalance Price [EUR/MWh] - SCA|FR',
       '- Imbalance Price [EUR/MWh] - SCA|FR', 'Status',
       'Total Imbalance [MWh] - SCA|FR', 'Difference', 'Situation',
       'Status.1'],
      dtype='object')


In [146]:
(df['+ Imbalance Price [EUR/MWh] - SCA|FR']==df['- Imbalance Price [EUR/MWh] - SCA|FR']).value_counts()

False    3004
True        1
dtype: int64

In [152]:
df.drop(['Status.1', 'Difference'], axis=1, inplace=True)

In [147]:
df.rename(columns={'+ Imbalance Price [EUR/MWh] - SCA|FR':'rebap_fr_pos_eur_mwh', 
                   '- Imbalance Price [EUR/MWh] - SCA|FR':'rebap_fr_neg_eur_mwh', 
                   'Total Imbalance [MWh] - SCA|FR':'imbalance_power_fr_mwh', 
                   'Status':'status',
                   'Situation':'sign',
                     }, inplace=True)

In [150]:
df['sign'] = [-1 if x=='Deficit' else 1 for x in df.sign]
df['imbalance_power_fr_mwh'] = df.imbalance_power_fr_mwh * df.sign
df.drop('sign', axis=1, inplace=True)

In [154]:
df.to_csv('data/entsoe_imbalance_fr.csv')

## `imbalance_pl`

Delay: 2.5h (10 time steps)

Preparation analogous.

## Final combination into 1 file 

Exclude fr, because data available only from May 29 2021. Finally, not used either as the countries have different delays.

In [ ]:
dfl = []
countries = ['at', 'be', 'dk', 'pl']
for country in countries:
    X = pd.read_csv('data/entsoe_imbalance_'+country+'.csv', 
                           parse_dates=['dt_start_utc'], 
                           index_col='dt_start_utc')
    dfl.append(X.drop('status', axis=1))
df = align(*dfl, prefixes=('' for country in countries))
df.to_csv('data/entsoe_imbalance.csv')

## Features

#### time 

- dayofyear
- dayofweek
- workday/holiday
- time

#### numerical ac

- crossborder flows:
    - original: from/to DK, NL, SE (6) 
    - names: DE_to_DK_power_mw, DK_to_DE_power_mw, DE_to_NL_power_mw, NL_to_DE_power_mw, DE_to_SE_power_mw, 
           SE_to_DE_power_mw, 
    - file: crossborder_flows_ts.csv
    - derive: total in, total out (2)

- control energy prices
    - original: control capacity prices, control energy prices and import/export volumes per sign, reserve type and area (total/Germany) (52)
    - names: all combinations of 
        - sign: NEG, POS
        - reserve types: MRL, SRL
        - area: total, Germany
        - capacity prices: min_capacity_price_eur_mw, average_capacity_price_eur_mw, marginal_capacity_price_eur_mw
        - energy prices: min_energy_price_eur_mwh, average_energy_price_eur_mwh, marginal_energy_price_eur_mwh 
        - import/export volume: germany_import_export_mw
    - file: regelleistung_aggr_4h.csv
    - derive: aggregate over reserve types and/or price types (capacity or energy) (52 --> 26 --> 13 --> 3)
    
- control power
    - original: total and Germany block demand per sign, reserve type and area (total/Germany) (48)
    - names: all combinations of 
        - sign: NEG, POS
        - reserve types: MRL, SRL
        - area: total_demand, Germany_block_demand
    - file: regelleistung_demand_4h.csv
    - derive: aggregate over reserve types and/or price types (capacity or energy) (8 --> 4 --> 2 --> 1)

- wind speed:
    - original: wind speed per Voronoi area (108)
    - names: area_x_wind_speed_ms with x = 1, 2, 4, ..., 109
    - file: wind_speed_ts.csv
    - derive: mean over areas (108 --> 1)



#### numerical fc
    
- forecast of renewables energy feed:
    - original: power in MW for carriers solar, wind offshore, wind onshore and areas (15)
    - names: all combinations of 
        - carriers: solar, wind_offshore, wind_onshore
        - areas: 50Hertz_power_mw, TTG_power_mw, DE_power_mw, DK_power_mw, DK1_power_mw
    - files: es_fc_solar_ts.csv, es_fc_wind_offshore_ts.csv, es_fc_wind_onshore_ts.csv
    - derive: aggregate over areas and/or carriers (15 --> 3 --> 1)

- forecast of total energy feed:
    - original: power in MW per area (5)
    - names: 50Hertz_power_mw, TTG_power_mw, DE_power_mw, DK_power_mw, DK1_power_mw
    - file: es_fc_total_load_ts.csv
    - derive: aggregate over areas (5 --> 1)

- extrapolation of renewables energy feed:
    - original: power in MW per carrier and area/total (12)
    - names: all combinations of
        - carriers solar, wind_onshore
        - areas fiftyhertz, tennet, amprion, transnetbw, total
        - plus wind_offshore with fiftyhertz, tennet, total
    - files: extrapol_solar_mw.csv, extrapol_wind_offshore_mw.csv, extrapol_wind_onshore_mw.csv 
    - derive: aggregate over carriers and/or area (12 --> 4 --> 1


#### numerical ac + fc

- EPEX day ahead contract prices:
    - original: 
        - ac: price per MWh for day ahead contracts in Germany (1)
        - fc: price per MWh from n = 1 ... 4 (... 45) days back of price per MWh for day ahead contracts in Germany (1 ... 45)
    - name: 
        - ac: epex_da_de_eur_mwh
        - fc: epex_da_de_eur_mwh = dict with keys n = 1, 2, 3, ... (days back)
    - files:
        - ac: epex_da_de.csv
        - fc: epex_da_prognosis.csv
    - derive: 
        - derivatives of fc wrt n (1, 2)
        - differences between last fc and ac (1)

- wind energy feed per feeding point:
    - original: power in kW (17 for ac and 22 for fc)
    - file: power_ac.csv, power_fc.csv
    - derive: 
        - aggregate over feeding points (17 --> 1, 22 --> 1)
        - take difference between fc and ac where applicable (34 --> 17 -->1)
    



#### categorical

- epex_da_de: sechs_h_regelung



#### delays in ac features

- cross border flows: 4h
- EPEX day ahead contract prices: 1 day (already implemented by reading off the newest prediction)

## Time series analysis


### Stationarity

tested rebap and its first and second derivative

result: ...


### ARIMA model



### fbprophet



### Result



### Dependence on month, dayofyear, weekday, hour, minute

In [ ]:
quantity = 'minute'
selection = None#'year'
plottype = 'l'
quantities = {'month':df.index.month,
              'weekday':df.index.weekday, 
              'dayofyear':df.index.dayofyear, 
              'hour':df.index.hour, 
              'time':df.index.time, 
              'minute':df.index.minute}
selections = {'weekday':df.index.weekday, 'month':df.index.month, 'year':df.index.year} 
plottypes = {'v':sns.violinplot, 'b':sns.boxplot, 'l':sns.lineplot}
fig = plt.figure(figsize=(20,10))
if quantity == 'minute':
    plt.xlabel('quarter of hour', fontsize=36)
    plt.ylabel('reBAP', fontsize=36)
    plt.xticks(ticks=[0,15,30,45], labels=['1st','2nd','3rd','4th'], fontsize=24)
    plt.yticks(fontsize=24)
if quantity == 'hour':
    plt.xlabel('daytime', fontsize=36)
    plt.ylabel('reBAP', fontsize=36)
    plt.xticks(ticks=range(25), fontsize=24)
    plt.yticks(fontsize=24)
if quantity == 'weekday':
    plt.xlabel('weekday', fontsize=36)
    plt.ylabel('reBAP', fontsize=36)
    plt.xticks(ticks=range(7), labels=['Mo','Tu','We','Th','Fr','Sa','Su'], fontsize=24)
    plt.yticks(fontsize=24)
if quantity == 'month':
    plt.xlabel('month', fontsize=36)
    plt.ylabel('reBAP', fontsize=36)
    plt.xticks(ticks=range(1,13),fontsize=24)
    plt.yticks(fontsize=24)
if quantity == 'dayofyear': plt.xticks(rotation=90)
if selection: 
    plottypes[plottype](x=quantities[quantity], y=df.rebap_eur_mwh, hue=selections[selection], legend='full')
    plt.savefig('images/'+quantity+'_per_'+selection+'_'+plottype+'.png', format='png', dpi=150);
else: 
    plottypes[plottype](x=quantities[quantity], y=df.rebap_eur_mwh)
    plt.savefig('images/'+quantity+'_'+plottype+'.png', format='png', dpi=150);

## Correlation for appropriately delayed numerical features

In [ ]:
df1 = pd.read_csv('data/imbalance_de.csv', parse_dates=True, index_col='dt_start_utc')
df1 = df1[df1.rebap_eur_mwh.mean()-3*df1.rebap_eur_mwh.std() <= df1.rebap_eur_mwh][df1.rebap_eur_mwh <= df1.rebap_eur_mwh.mean()+3*df1.rebap_eur_mwh.std()]
df1['power_mw'] = df1.power_mw.shift(periods=5)
df1.dropna(axis=0, inplace=True)

df2 = pd.read_csv('data/power_ac.csv', parse_dates=True, index_col='dt_start_utc')

df = align(df1, df2, prefixes=('',''), method='forward')

sorted(df1.corr(method='pearson')['rebap_eur_mwh'].items(), key = lambda x : x[1], reverse=True)[1:]

### Features to be included

month (D)

weekday (D) (from lineplot)

hour (D)

minute (D)
(or linear segments 0-3, 3-6, 6-11:30, 11:30-15, 15-17, 17-19, 19-21, 21-23) (from lineplot)

epex_da_de_eur_mwh

epex_da_prognosis: newest_fc, second_newest_fc

es_fc_total_load: total_power_mw

es_fc_wind_offshore: total_power_mw

es_fc_wind_onshore: total_power_mw

power_ac: power_act_206, 21, 84, 308, 293, total_power_kw

power_fc: power_fc_47, 206, 24, 233, 21, 196, 308, 293, total_power_kw

power_fc - power_ac: diff_206, 21, 308, 293

wind speed: area_2, 1, 87, 102

## Plan


### Time

- analyze stationarity, trend + seasonality (seasonal_decompose)
- smoothing, rolling window
- autocorrelation

--> model pure time dependence (ARIMA, prophet)


### Features

- add weekday, businessday/holiday, dayofyear, hour to the features
- write table with features and type of values: ac or fc
- shift ac values to time they are available
- idea: use differences between ac and fc values if applicable
- further shift all features by desired advance time of prediction

--> model rebap by any regressor
--> model rebap sign by any classifier


### Theory

- multivariate time series analysis
